In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score

In [4]:
# Load the model and tokenizer
model_name = "Olivernyu/finetuned_bert_base_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Load the train.csv file and extract a portion (300 samples) of the data for testing
df = pd.read_csv('train.csv')
test_data = df.sample(n=300, random_state=42)  # You can change the random_state for different samples

# Extract the comment text and the true labels for classification
test_comments = test_data['comment_text'].tolist()
true_labels = test_data[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values

# Preprocess the test data
inputs = tokenizer(test_comments, return_tensors='pt', padding=True, truncation=True)




In [5]:
import torch
# Run the model on the test data and obtain predictions
with torch.no_grad():
    outputs = model(**inputs)
    # Apply sigmoid function to convert logits to probabilities
    probabilities = torch.sigmoid(outputs.logits).numpy()

# Apply a threshold to convert probabilities to predicted labels (0 or 1)
threshold = 0.5
predicted_labels = (probabilities >= threshold).astype(int)

# Calculate accuracy for each category
for i, category in enumerate(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']):
    accuracy = accuracy_score(true_labels[:, i], predicted_labels[:, i])
    print(f"Accuracy for {category}: {accuracy:.4f}")

Accuracy for toxic: 0.9867
Accuracy for severe_toxic: 0.9933
Accuracy for obscene: 0.9900
Accuracy for threat: 1.0000
Accuracy for insult: 0.9867
Accuracy for identity_hate: 0.9933
